In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
directory_in = './file/3.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input = df_input[df_input['Vehicle Url'] != ''].reset_index(drop=True)

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：92



,No,Type,Part Number,Brand,Description,Application Summary,OEM,Picture,Url,Json_Src,Vehicle Url,Json_Specification
0,1,Accessory Drive Belt Idler Pulley,419-5002,Dorman - HD Solutions,Idler Pulley (Pulley Only),"FORD MOTOR COMPANY 2017-93, GENERAL MOTORS 201...",10239670;1753054L10;10239930;11287509508;12456...,,https://www.dormanproducts.com/p-81798-419-500...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Belt Type"": ""Serpentine""}, ""1"": {""Colo..."
1,2,Air Conditioning Bypass Pulley,34150,Dorman - HELP! - Boxed,Air Conditioning Bypass Pulley,Ford 1997-90,F0TZ-8678A,,https://www.dormanproducts.com/p-19727-34150.a...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Compressor Type"": ""Fs10""}, ""1"": {""Inst..."
2,3,Air Conditioning Bypass Pulley,34151,Dorman - HELP! - Boxed,Air Conditioning Bypass Pulley,"Ford Taurus 1995-88, Mercury Sable 1995-88",E7DZ-8678B,,https://www.dormanproducts.com/p-27239-34151.a...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Compressor Type"": ""Fs10""}, ""1"": {""Inst..."
3,4,Air Conditioning Bypass Pulley,34152,Dorman - HELP! - Boxed,Air Conditioning Bypass Pulley,"Chevrolet 1995-87, GMC 1995-87, Oldsmobile 199...",10118713,,https://www.dormanproducts.com/p-6941-34152.as...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Compressor Type"": ""Ltr4""}, ""1"": {""Inst..."
4,5,Air Conditioning Bypass Pulley,34153,Dorman - HELP! - Boxed,Air Conditioning Bypass Pulley,"Buick 1995-92, Chevrolet 2002-90, Oldsmobile 1...",,,https://www.dormanproducts.com/p-7392-34153.as...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Compressor Type"": ""Fs10""}, ""1"": {""Inst..."
...,...,...,...,...,...,...,...,...,...,...,...,...
87,89,Air Conditioning Bypass Pulley,34863,Dorman - HELP! - Boxed,Air Conditioning Bypass Pulley,"Jeep Commander 2007-06, Jeep Grand Cherokee 20...",77361,,https://www.dormanproducts.com/p-131746-34863....,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Color"": ""Natural""}, ""1"": {""Compressor ..."
88,90,Air Conditioning Bypass Pulley,34868,Dorman - HELP! - Boxed,Air Conditioning Bypass Pulley,"Toyota Camry 2001-86, Toyota Celica 1999-88, T...",57398,,https://www.dormanproducts.com/p-131983-34868....,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Color"": ""Natural""}, ""1"": {""Compressor ..."
89,91,Air Conditioning Bypass Pulley,34869,Dorman - HELP! - Boxed,Air Conditioning Bypass Pulley,Nissan Altima 2001-00,67449,,https://www.dormanproducts.com/p-131747-34869....,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Color"": ""Natural""}, ""1"": {""Compressor ..."
90,92,Air Conditioning Bypass Pulley,34890,Dorman - HELP! - Boxed,Air Conditioning Bypass Pulley,"Hyundai Santa Fe 2006-01, Hyundai Sonata 2005-...",,,https://www.dormanproducts.com/p-126453-34890....,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Color"": ""Natural""}, ""1"": {""Compressor ..."


In [4]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                                     1
 Type                                   Accessory Drive Belt Idler Pulley
 Part Number                                                     419-5002
 Brand                                              Dorman - HD Solutions
 Description                                   Idler Pulley (Pulley Only)
 Application Summary    FORD MOTOR COMPANY 2017-93, GENERAL MOTORS 201...
 OEM                    10239670;1753054L10;10239930;11287509508;12456...
 Picture                                                                 
 Url                    https://www.dormanproducts.com/p-81798-419-500...
 Json_Src               {"0": "https://static.dormanproducts.com/image...
 Vehicle Url            https://www.dormanproducts.com/itemdetailapp.a...
 Json_Specification     {"0": {"Belt Type": "Serpentine"}, "1": {"Colo...
 Name: 0, dtype: object)

In [5]:
import time

list_dict_correct_temp, page = [], 0
while True:

    count_retry = 0
    while True:
        count_retry += 1
        print(page + 1, count_retry)
        try:
            url_request = f'''{crawler_series['Vehicle Url']}&num=50&start={page * 50}'''

            resp = requests.get(url_request,
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(5, 15))

            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                if html.xpath('//div[@class="detail-app-pager"]'):
                    if page == 0:
                        with open('./vehicle.html', 'w', encoding='UTF-8') as file:
                            file.write(str(soup))
                    break
        except KeyboardInterrupt:
            break
        except:
            time.sleep(3)
            continue

    # = = = = = = = = = = = = = = =

    next_ = html.xpath('//a[@id="pagingBottom_nextButton"]')
    if page == 0 and next_ and next_[0].xpath('./@href')[0].strip() == 'Search/#':
        break

    # = = = = = = = = = = = = = = =

    list_text, list_tr = [text.strip() for text in html.xpath('//th[@scope="col"]/text()')], html.xpath('//tr[@class="detail-app-row"]')

    # = = = = = = = = = = = = = = =

    list_dict_correct_temp_temp = [{'Vehicle Url': crawler_series['Vehicle Url'],
                                    'Page': page + 1,
                                    'Row': i + 1} for i in range(len(list_tr))]

    for i, tr in enumerate(list_tr):
        for j, text in enumerate(list_text):
            list_dict_correct_temp_temp[i][text] = tr.xpath(f'./td[{j+1}]/text()')[0].strip()

    list_dict_correct_temp.extend(list_dict_correct_temp_temp)

    # = = = = = = = = = = = = = = =

    if not next_:
        break

    # = = = = = = = = = = = = = = =

    page += 1

list_dict_correct_temp

1 1
1 2
1 3
2 1
3 1
4 1
4 2
5 1
5 2
6 1
7 1
8 1
9 1
10 1
11 1
12 1
13 1
14 1
15 1


[{'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=81798&PartType=Accessory%20Drive%20Belt%20Idler%20Pulley',
  'Page': 1,
  'Row': 1,
  'Year': '2017',
  'Make': 'Ford',
  'Model': 'EcoSport (Mexico)',
  'Configuration': 'L4 122 2.0L (1999cc)',
  'Positions': '',
  'Application Notes': ''},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=81798&PartType=Accessory%20Drive%20Belt%20Idler%20Pulley',
  'Page': 1,
  'Row': 2,
  'Year': '2016',
  'Make': 'Chevrolet',
  'Model': 'Silverado 1500',
  'Configuration': 'V6 262 4.3L (4300cc); Engine Desg. LU3',
  'Positions': 'Air Conditioning Bypass',
  'Application Notes': 'wo/Air Conditioning'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=81798&PartType=Accessory%20Drive%20Belt%20Idler%20Pulley',
  'Page': 1,
  'Row': 3,
  'Year': '2016',
  'Make': 'Ford',
  'Model': 'EcoSport (Mexico)',
  'Configuration': 'L4 122 2.0L (1999cc)',
  'Positions': '',
  'App

In [6]:
list_dict_correct_temp

[{'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=81798&PartType=Accessory%20Drive%20Belt%20Idler%20Pulley',
  'Page': 1,
  'Row': 1,
  'Year': '2017',
  'Make': 'Ford',
  'Model': 'EcoSport (Mexico)',
  'Configuration': 'L4 122 2.0L (1999cc)',
  'Positions': '',
  'Application Notes': ''},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=81798&PartType=Accessory%20Drive%20Belt%20Idler%20Pulley',
  'Page': 1,
  'Row': 2,
  'Year': '2016',
  'Make': 'Chevrolet',
  'Model': 'Silverado 1500',
  'Configuration': 'V6 262 4.3L (4300cc); Engine Desg. LU3',
  'Positions': 'Air Conditioning Bypass',
  'Application Notes': 'wo/Air Conditioning'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=81798&PartType=Accessory%20Drive%20Belt%20Idler%20Pulley',
  'Page': 1,
  'Row': 3,
  'Year': '2016',
  'Make': 'Ford',
  'Model': 'EcoSport (Mexico)',
  'Configuration': 'L4 122 2.0L (1999cc)',
  'Positions': '',
  'App

In [7]:
list_dict_correct.extend(list_dict_correct_temp)

list_dict_correct

[{'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=81798&PartType=Accessory%20Drive%20Belt%20Idler%20Pulley',
  'Page': 1,
  'Row': 1,
  'Year': '2017',
  'Make': 'Ford',
  'Model': 'EcoSport (Mexico)',
  'Configuration': 'L4 122 2.0L (1999cc)',
  'Positions': '',
  'Application Notes': ''},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=81798&PartType=Accessory%20Drive%20Belt%20Idler%20Pulley',
  'Page': 1,
  'Row': 2,
  'Year': '2016',
  'Make': 'Chevrolet',
  'Model': 'Silverado 1500',
  'Configuration': 'V6 262 4.3L (4300cc); Engine Desg. LU3',
  'Positions': 'Air Conditioning Bypass',
  'Application Notes': 'wo/Air Conditioning'},
 {'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=81798&PartType=Accessory%20Drive%20Belt%20Idler%20Pulley',
  'Page': 1,
  'Row': 3,
  'Year': '2016',
  'Make': 'Ford',
  'Model': 'EcoSport (Mexico)',
  'Configuration': 'L4 122 2.0L (1999cc)',
  'Positions': '',
  'App

In [8]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                                     1
 Type                                   Accessory Drive Belt Idler Pulley
 Part Number                                                     419-5002
 Brand                                              Dorman - HD Solutions
 Description                                   Idler Pulley (Pulley Only)
 Application Summary    FORD MOTOR COMPANY 2017-93, GENERAL MOTORS 201...
 OEM                    10239670;1753054L10;10239930;11287509508;12456...
 Picture                                                                 
 Url                    https://www.dormanproducts.com/p-81798-419-500...
 Json_Src               {"0": "https://static.dormanproducts.com/image...
 Vehicle Url            https://www.dormanproducts.com/itemdetailapp.a...
 Json_Specification     {"0": {"Belt Type": "Serpentine"}, "1": {"Colo...
 Request_Url            https://www.dormanproducts.com/itemdetailapp.a...
 Name: 0, dtype: object]

In [9]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Vehicle Url', 'Page', 'Row'],
                                                             ascending=[True, True, True],
                                                             ignore_index=True)
    df_correct['Year'] = df_correct['Year'].astype(int)
    df_correct.to_excel('./test_vehicle.xlsx', index=False)

df_correct

,Vehicle Url,Page,Row,Year,Make,Model,Configuration,Positions,Application Notes
0,https://www.dormanproducts.com/itemdetailapp.a...,1,1,2017,Ford,EcoSport (Mexico),L4 122 2.0L (1999cc),,
1,https://www.dormanproducts.com/itemdetailapp.a...,1,2,2016,Chevrolet,Silverado 1500,V6 262 4.3L (4300cc); Engine Desg. LU3,Air Conditioning Bypass,wo/Air Conditioning
2,https://www.dormanproducts.com/itemdetailapp.a...,1,3,2016,Ford,EcoSport (Mexico),L4 122 2.0L (1999cc),,
3,https://www.dormanproducts.com/itemdetailapp.a...,1,4,2015,Chevrolet,Silverado 1500,V6 262 4.3L,Air Conditioning Bypass,wo/Air Conditioning
4,https://www.dormanproducts.com/itemdetailapp.a...,1,5,2015,Ford,EcoSport (Mexico),L4 122 2.0L (1999cc),,
...,...,...,...,...,...,...,...,...,...
698,https://www.dormanproducts.com/itemdetailapp.a...,14,49,1994,Mazda,Navajo,V6 245 4.0L,Grooved Pulley,
699,https://www.dormanproducts.com/itemdetailapp.a...,14,50,1993,Chevrolet,P30,V6 262 4.3L,Grooved Pulley,wo/Air Conditioning; 73 mm O.D. Pulley
700,https://www.dormanproducts.com/itemdetailapp.a...,15,1,1992,Chevrolet,P30,V6 262 4.3L,Grooved Pulley,wo/Air Conditioning; 73 mm O.D. Pulley
701,https://www.dormanproducts.com/itemdetailapp.a...,15,2,1991,Chevrolet,P30,V6 262 4.3L,Grooved Pulley,wo/Air Conditioning; 73 mm O.D. Pulley


In [10]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_vehicle_error.xlsx', index=False)

df_error

,No,Type,Part Number,Brand,Description,Application Summary,OEM,Picture,Url,Json_Src,Vehicle Url,Json_Specification,Request_Url
0,1,Accessory Drive Belt Idler Pulley,419-5002,Dorman - HD Solutions,Idler Pulley (Pulley Only),"FORD MOTOR COMPANY 2017-93, GENERAL MOTORS 201...",10239670;1753054L10;10239930;11287509508;12456...,,https://www.dormanproducts.com/p-81798-419-500...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Belt Type"": ""Serpentine""}, ""1"": {""Colo...",https://www.dormanproducts.com/itemdetailapp.a...
